<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importar-Librerías" data-toc-modified-id="Importar-Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importar Librerías</a></span></li><li><span><a href="#Conexión-con-la-API-de-Meli" data-toc-modified-id="Conexión-con-la-API-de-Meli-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Conexión con la API de Meli</a></span><ul class="toc-item"><li><span><a href="#Acceso-al-Token" data-toc-modified-id="Acceso-al-Token-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Acceso al Token</a></span></li><li><span><a href="#Mercado-Libre-API---Search" data-toc-modified-id="Mercado-Libre-API---Search-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Mercado Libre API - Search</a></span></li><li><span><a href="#Construcción-del-Dataset-de-Productos" data-toc-modified-id="Construcción-del-Dataset-de-Productos-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Construcción del Dataset de Productos</a></span></li></ul></li></ul></div>

# Pulling Data - Mercado Libre API

El propósito de este notebook es realizar el primer paso del reto de DS y ML propuesto por el equipo Meli: Obtener los datos de ciertos productos a través de la [API](https://api.mercadolibre.com/) de Mercado Libre.

En este notebook se establece la conexión con la API y se obtiene através del buscador de Mercado Libre ciertos productos para construir un dataset que será usuado tanto para la parte exploratoria como la del modelo de ML.

## Importar Librerías

In [1]:
# Cambia el ancho de las celdas
from IPython.display import HTML, display
display(HTML(data="""<style>div#notebook-container{width: 60%;}div#menubar-container{width: 65%;} div#maintoolbar-container{width: 99%;}</style>"""))

In [45]:
# Para el acceso, autorizacion y token de ApiMeli
import urllib
from __future__ import print_function
import time
import meli
from meli.rest import ApiException
from pprint import pprint
import json
import os

# Procesmaiento de datos
import pandas as pd
import numpy as np

## Conexión con la API de Meli

### Acceso al Token

In [3]:
# Variables de app_id, redirect_uri y tipo de response
RESPONSE_TYPE = 'code'
CLIENT_ID = '4859773784768141'
REDIRECT_URI = 'http://localhost/'

# Devuelve el link al codigo de autorizacion
params = urllib.parse.urlencode({'response_type': RESPONSE_TYPE, 'client_id': CLIENT_ID, 
                                 'redirect_uri':REDIRECT_URI})
f = urllib.request.urlopen("https://auth.mercadolibre.com.co/authorization?%s" % params)
print('EL LINK ES:\t', f.geturl())

EL LINK ES:	 https://www.mercadolibre.com/jms/mco/lgz/login/?go=https%3A%2F%2Fauth.mercadolibre.com.co%2Fauthorization%3Fclient_id%3D4859773784768141%26redirect_uri%3Dhttp%253A%252F%252Flocalhost%252F%26response_type%3Dcode&platform_id=ml&application_id=4859773784768141


In [4]:
# Variable con el secret_key
CLIENT_SECRET = '51ThrmmqvO2vazuX7eFt4zEz9kT77h0t'

# Genera el token, refresh_token para el uso de la api
configuration = meli.Configuration(host = "https://api.mercadolibre.com")

# Contexto con una instancia de la API cliente
with meli.ApiClient() as api_client:
# Crea una instancia con la clase API
    api_instance = meli.OAuth20Api(api_client)
    grant_type = 'authorization_code' # Tipo de Grant
    client_id = CLIENT_ID # App ID
    client_secret = CLIENT_SECRET # Secret Key
    redirect_uri = REDIRECT_URI # Redirect URI
    code = 'TG-6048dc75f8927a0007ed423a-297551045' # El codigo de autorizacion
try:
    # Solicita el token de acceso
    api_response = api_instance.get_token(grant_type=grant_type,
                                          client_id=client_id,
                                          client_secret=client_secret,
                                          redirect_uri=redirect_uri,
                                          code=code)
    print('API RESPONSE: \n')
    pprint(api_response)
except ApiException as e:
    print("Exception when calling OAuth20Api->get_token: %s\n" % e)

API RESPONSE: 

{'access_token': 'APP_USR-4859773784768141-031014-3ca6c6d6251626da0542ce1498ce43b3-297551045',
 'expires_in': 21600,
 'refresh_token': 'TG-6048dc800b7483000807e69f-297551045',
 'scope': 'offline_access read write',
 'token_type': 'bearer',
 'user_id': 297551045}


In [5]:
# Se guarda el token en una variable
TOKEN = api_response['access_token']
REFRESH_TOKEN = api_response['refresh_token']

### Mercado Libre API - Search

In [6]:
# Se obtienen la lista de categorias (id y name) de Meli
with meli.ApiClient() as api_client:
    api_instance = meli.RestClientApi(api_client)
    resource = 'sites/MCO/categories' # A resource example like items, search, category, etc.
    access_token = TOKEN # El token de acceso

try:
    # Obtiene las categoias de meli
    meli_cats = api_instance.resource_get(resource, access_token)
except ApiException as e:
    print("Exception when calling RestClientApi->resource_get: %s\n" % e)

In [33]:
# Se obtienen la lista de categorias (id y name) de Meli
limit = 50 
items_per_category = 500
repetitions = items_per_category//limit

# Si ya se halaron los datos asigne a esta variable False
obtener_data = False
if obtener_data:
    # Lista con los resultados de las busquedas
    searchs = []
    # Obtiene los 500 primeros items (por relevancia) para cada categoria
    for cat in meli_cats:
        offset = 0
        for r in range(repetitions):
            with meli.ApiClient() as api_client:
                api_instance = meli.RestClientApi(api_client)
                resource = 'sites/MCO/search?category={}&offset={}'.format(cat['id'], offset)
                access_token = TOKEN # El token de acceso
                offset += limit
            try:
                # Obtiene las search para la categoria de
                api_response = api_instance.resource_get(resource, access_token)
                searchs.append(api_response)
            except ApiException as e:
                print("Exception when calling RestClientApi->resource_get: %s\n" % e)
        print('PROCESADA LA CATEGORIA: {}'.format(cat['name'].upper()))

PROCESADA LA CATEGORIA: ACCESORIOS PARA VEHÍCULOS
PROCESADA LA CATEGORIA: AGRO
PROCESADA LA CATEGORIA: ALIMENTOS Y BEBIDAS
PROCESADA LA CATEGORIA: ANIMALES Y MASCOTAS
PROCESADA LA CATEGORIA: ANTIGÜEDADES Y COLECCIONES
PROCESADA LA CATEGORIA: ARTE, PAPELERÍA Y MERCERÍA
PROCESADA LA CATEGORIA: BEBÉS
PROCESADA LA CATEGORIA: BELLEZA Y CUIDADO PERSONAL
PROCESADA LA CATEGORIA: BOLETAS PARA ESPECTÁCULOS
PROCESADA LA CATEGORIA: CÁMARAS Y ACCESORIOS
PROCESADA LA CATEGORIA: CARROS, MOTOS Y OTROS
PROCESADA LA CATEGORIA: CELULARES Y TELÉFONOS
PROCESADA LA CATEGORIA: COMPUTACIÓN
PROCESADA LA CATEGORIA: CONSOLAS Y VIDEOJUEGOS
PROCESADA LA CATEGORIA: DEPORTES Y FITNESS
PROCESADA LA CATEGORIA: ELECTRODOMÉSTICOS
PROCESADA LA CATEGORIA: ELECTRÓNICA, AUDIO Y VIDEO
PROCESADA LA CATEGORIA: HERRAMIENTAS Y CONSTRUCCIÓN
PROCESADA LA CATEGORIA: HOGAR Y MUEBLES
PROCESADA LA CATEGORIA: INDUSTRIAS Y OFICINAS
PROCESADA LA CATEGORIA: INMUEBLES
PROCESADA LA CATEGORIA: INSTRUMENTOS MUSICALES
PROCESADA LA CATEGORIA: J

In [34]:
# Se salva en un json cada una de las busquedas con sus parametros (categoria y offset) si obtener_data es True
if obtener_data:
    for search in searchs:
        # Salva search
        cat = search['filters'][0]['values'][0]['id']
        offset_parameter = search['paging']['offset']
        with open('../Data/Searchs/CAT_{}__OFFSET_{}.json'.format(cat, offset_parameter), 'w') as fp:
            json.dump(search, fp)

El 10 de Marzo de 2021 a las 10:15 horas se hizo una búsqueda de los __500__ primeros ítems que aparecen para cada categoría (serán los los 500 más relevantes debido a que ese es el orden por default del marketplace) en el [Mercado Libre Colombia](https://www.mercadolibre.com.co/) por medio de los filtros de `category` y `offset`.

La razón por la cual no se hizo ningún filtro relacionado al descuento es porque me parece importante entender cual es el porcentaje de productos de cada categoría que tienen descuento.

In [41]:
# De ser necesario lee los jsons guardados
# Lealos en vez de 
leer_jsons = True
if leer_jsons:
    searchs = []
    path = '../Data/Searchs/'
    json_files = os.listdir(path)
    for file in json_files:
        with open(path + file) as json_file:
            searchs.append(json.load(json_file))

### Construcción del Dataset de Productos

In [42]:
# Se define una lista con las variables asociadas a un producto (es decir no del vendedor 
# o del shipping de el producto por ejemplo)
item_keys = ['id', 'site_id', 'title', 'price', 'sale_price',
             'currency_id', 'available_quantity', 'sold_quantity', 'buying_mode',
             'listing_type_id', 'stop_time', 'condition', 'permalink', 'thumbnail', 
             'thumbnail_id', 'accepts_mercadopago', 'original_price', 
             'category_id', 'official_store_id', 'domain_id', 'catalog_product_id',
             'order_backend', 'use_thumbnail_id']

In [43]:
# En este doble for se recorre los resultados de cada busqueda y
# se crea una base con productos (la cual se usara tanto para la parte exploratoria 
# como para el modelo)
products = pd.DataFrame()
errores = 0
total_de_productos = 0
for search in searchs:
    for idx, result in enumerate(search['results']):
        # Se pone un try debido a que puede haber informacion incompleta 
        # de algunos productos (shipping, seller, etc)
        try:
            ## Item vars: En esta parte se integra la info del item especificamente
            vars_dict = {'item_'+key: result[key] for key in item_keys}
            vars_dict['search_category_id'] = search['filters'][0]['values'][0]['id']
            vars_dict['search_category_name'] = search['filters'][0]['values'][0]['name']
            vars_dict['search_offset'] = search['paging']['offset']

            ## Seller vars: En esta parte se integra la info del vendedor del item especificamente
            vars_dict['seller_id'] = result['seller']['id']

            # Reputation and transaction vars
            vars_dict['seller_rep_transactions_total'] = result['seller']['seller_reputation']['transactions']['total']
            vars_dict['seller_rep_transactions_canceled'] = result['seller']['seller_reputation']['transactions']['canceled']
            vars_dict['seller_rep_rating_neg'] = result['seller']['seller_reputation']['transactions']['ratings']['negative']
            vars_dict['seller_rep_rating_pos'] = result['seller']['seller_reputation']['transactions']['ratings']['positive']
            vars_dict['seller_rep_rating_neu'] = result['seller']['seller_reputation']['transactions']['ratings']['neutral']
            vars_dict['seller_transactions_completed'] = result['seller']['seller_reputation']['transactions']['completed']
            vars_dict['seller_status'] = result['seller']['seller_reputation']['power_seller_status']

            # Metrics vars
            vars_dict['seller_metrics_claims_rate'] = result['seller']['seller_reputation']['metrics']['claims']['rate']
            vars_dict['seller_metrics_claims_value'] = result['seller']['seller_reputation']['metrics']['claims']['value']
            vars_dict['seller_metrics_claims_period'] = result['seller']['seller_reputation']['metrics']['claims']['period']
            vars_dict['seller_metrics_delay_rate'] = result['seller']['seller_reputation']['metrics']['delayed_handling_time']['rate']
            vars_dict['seller_metrics_delay_value'] = result['seller']['seller_reputation']['metrics']['delayed_handling_time']['value']
            vars_dict['seller_metrics_delay_period'] = result['seller']['seller_reputation']['metrics']['delayed_handling_time']['period']
            vars_dict['seller_metrics_sales_period'] = result['seller']['seller_reputation']['metrics']['sales']['period']
            vars_dict['seller_metrics_sales_completed'] = result['seller']['seller_reputation']['metrics']['sales']['completed']
            vars_dict['seller_metrics_cancellations_period'] = result['seller']['seller_reputation']['metrics']['cancellations']['period']
            vars_dict['seller_metrics_cancellations_rate'] = result['seller']['seller_reputation']['metrics']['cancellations']['rate']
            vars_dict['seller_metrics_cancellations_value'] = result['seller']['seller_reputation']['metrics']['cancellations']['value']

            # Level id var (ej Platinum)
            vars_dict['seller_level_id'] = result['seller']['seller_reputation']['level_id']

            ## Shipping vars: En esta parte se integra la info del shipping del item especificamente
            vars_dict['shipping_free'] = result['shipping']['free_shipping']
            vars_dict['shipping_logistic_type'] = result['shipping']['logistic_type']
            vars_dict['shipping_pick_up'] = result['shipping']['store_pick_up']

            ## Adress vars
            vars_dict['adress_state_id'] = result['address']['state_id']
            product = pd.DataFrame(vars_dict, index=[0])
            products = pd.concat([products, product], axis=0)
        except:
            errores += 1
        total_de_productos += 1
        
# Se imprimen las dimensiones de la base de productos construida        
products.reset_index(drop=True, inplace=True)
print('LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: {}'.format(products.shape))

LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: (15376, 50)


In [46]:
porcentaje_de_errores = np.round(errores/total_de_productos*100,2)
print('EL PORCENTAJE DE ERRORES PRESENTADOS FUE DE {}%, EN TOTAL {} PRODUCTOS'.format(porcentaje_de_errores, errores))

EL PORCENTAJE DE ERRORES PRESENTADOS FUE DE 0.67%, EN TOTAL 104 PRODUCTOS


In [47]:
# Se crea una tabla de duplicados por el item_id
duplicados_por_id = products.loc[products['item_id'].duplicated(keep=False)]

In [48]:
# Se remueven los duplicados
products.drop_duplicates(subset=['item_id'], keep='first', inplace=True)
print('LAS DIMENSIONES DE LA BASE DE PRODUCTOS DESPUES DE REMOVER DUPLICADOS SON: {}'.format(products.shape))

LAS DIMENSIONES DE LA BASE DE PRODUCTOS DESPUES DE REMOVER DUPLICADOS SON: (14574, 50)


In [49]:
# Se guarda el dataset en un .csv (en caso de que haya un problema con el notebook,
# se apague el compu o algo asi, ya se tiene el dataset de datos)
products.to_csv('../Data/Datasets/PRODUCTOS.csv', index=False)

# Fin del Documento